<a href="https://colab.research.google.com/github/Unaifrixa9/TFG/blob/main/Pruebas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from time import time
import math
import itertools

'''
    Nombre de la función: report
    Funcionalidad: Función que crea los nodo de los Chronicles
    Parametros:
        - iset: Itemset que se combinandolos el soporte es mayor que el minimo
        - pexs: Items que son extensiones perfectas de iset
        - supp: Soporte del Chronicle
        - minLen: Longitud mínima
'''
def report (iset, pexs, supp, minLen):
    if not pexs:
        #No perfect extensions
        iteNames=[]
        #Comprobamso que la longitud del iset sea mayor o igual que minLen
        if len(iset) >= minLen:
            for i in iset: iteNames.append(i)
            valid = False
            #Bucle para comprobar que no se creen chornicles duplicados
            for v in val.values():
                for w in v:
                    if w in iset:
                        iset.remove(w)
                    else:
                        break
                if len(iset) == 0:
                    valid = True
                    break
            if valid == True:
                itemsets[frozenset(iteNames)]=supp
    else:
        #Report recursivo para comprobar todas las combinaciones
        report(iset+[pexs[0]], pexs[1:], supp, minLen)
        report(iset,           pexs[1:], supp, minLen)

'''
    Nombre de la función: Recurse
    Funcionalidad: Función recursiva para la creación de las combinaciones de nodos frecuentes
    Parametros:
        - tadb: Lista que contine todos los items con soporte mayor que minSup, el soporte de este item
            y las transacciones en las que esta el item.
        - iset: Itemset que se combinandolos el soporte es mayor que el minimo
        - pexs: Items que son extensiones perfectas de iset
        - minSup: Soporte mínimo
        - minLen: Longitud minima
'''
def recurse (tadb, iset, pexs, minSup, minLen):
    for k in range(len(tadb)):
        #Obtenemos los datos de un nodo
        s,i,t = tadb[k]  #s: number of transactions satisfied; i: item; t: transactions
        proj = []; xpxs = []

        #Obtenemos los datos de los siguientes
        for r,j,u in tadb[k+1:]: #r: number of transactions satisfied; j: item; u: transactions
            #Combine two item(sets)
            zzz = u

            #Combinar transacciones donde aparecen i y j
            u = u & t

            #Suma de los dos
            r = sum([w for x,w in u])

            #Si la suma r > que el soporte del nodo i se pueden combinar
            if r >= s:
                xpxs.append(j) #Item j is a perfect extension of i (La combinación tiene un soporte igual o mayor)
            #Sino si la suma r > que el sopote minimo creamos proyeccion de r
            elif r >= minSup:   #Combinación de j con itemset i no es perfect extension, pero si que es mayor que el soporte
                proj.append([r,j,u]) #Projection

        xpxs = pexs +xpxs  #Perfect extensions
        xset = iset +[i]   #Resulting itemset

        recurse(proj, xset, xpxs, minSup, minLen) #Recursively considering the projections
        report(xset, xpxs, s, minLen) #Print the partial result

'''
    Nombre de la función: LCM
    Funcionalidad: Creación de los Frequent Chronicle
    Parametros:
        - Tracts: Lista de frozenset con los nodos de cada registra en versión numerica
        - minSup: Soporte míninimo que deben tener los Frequente Chronicle
        - miLen: Longitud míninimo que deben tener los Frequente Chronicle
'''
def LCM (tracts, minSup, minLen):
    tadb = {}
    #Database as frozensets. Duplicated transactions are counted
    for t in [frozenset(t) for t in tracts]:
        #print(t)
        if t in tadb: tadb[t] += 1
        else:         tadb[t]  = 1
    #print("tasb -->", tadb)
    tracts = tadb.items()
    items  = set().union(*[t for t,w in tracts]) #Take the items from Database
    tadb   = dict([(i,[]) for i in items])


    #for each item, transactions that contain such an item
    for t in tracts:
        for i in t[0]:
            tadb[i].append(t)

    #Set including: item, number of transactions having the item, list of transactions having the item
    tadb = [[sum([w for t,w in tadb[i]]), i, set(tadb[i])] for i in tadb]
    tadb = [t for t in tadb if t[0] >= minSup] #Keep just those having the support satisfied


    recurse(tadb, [], [], minSup, minLen)

'''
    Nombre de la funcion: loadBD
    Funcionalidad: Lectura de la base de datos
    Parametros: 
        - File -> Cadena de caracteres con el nombre del fichero donde esta la base de datos
    Nota: La función crea varias estructura de datos a partir de los datos.
    Return:
        - data: Lista de diccionarioscon keys = Nodos del registro, y los values = Tiempos de cada nodo
        - flat_list: Lista de frozenset con los nodos de cada registra en versión numerica
        - val: Diccionario con las correspondecias letra-número de los nodos
        - dAux: Lista de los registros con los nodos en modo número.
'''
def loadBD(file):
    data = []
    index = 0
    val = {}
    dAux = []
    flat_list=[]
    #Crear lista con diccionario para cada registro
    with open(file) as f:
        for line in f.readlines():
            reg = {}
            for i in line.split(':'):
                event = i.split(' ')
                if event[0] in reg: reg[event[0]].append(int(event[1]))
                else: reg[event[0]] = [int(event[1])]
            data.append(reg)
            d = {}


            #Creación de la val
            for key, value in reg.items():
                lenValue = len(value) #Nº de veces que hay un nodo en cada registro
                if key not in val:  #Si el nodo no esta en val{}
                    l = []
                    for i in range(lenValue):
                        l.append(index) #Guardar en l index de cada nodo
                        index+=1
                    d[key] = l
                    val[key] = l
                elif len(val[key]) < lenValue:
                    l = val[key]
                    for i in range(lenValue-len(val[key])):
                        l.append(index)
                        index+=1
                    val[key] = l
                    d[key] = l
                else:
                    for i in range(lenValue):
                        d[key] = val[key][:lenValue]


            #Creación del flat_list
            tracts=[j for j in d.values()]
            flat_list.append(frozenset(sum(tracts, [])))

            #Creación de dAux
            d2 = {}
            for k,v in reg.items():
                it = 0
                vK = val[k]
                for z in v:
                    d2[vK[it]] = z
                    it+=1
            dAux.append(d2)
            

    f.close()
    
    return data,flat_list,val,dAux

'''
    Nombre función: duplicatedInList
    Funcionalidad: Comprueba si en una lista hay datos duplicados
    Parametro:
        - xs: Lista de números
    Return:
        - True si hay datos duplucados
        - False si no los hay
'''
def duplicatedInList(xs):
    s = set()
    return any(x in s or s.add(x) for x in xs)

'''
    Nombre de la función: recurseCombi
    Funcionalidad: Calcula todas las combinaciones de tiempos para los nodos dados para un unico registro de la base de datos
    Parametros:
        - re: Combinación de los nodos en cada paso recursivo
        - l: Conjunto de nodos con los que nos falta hacer la combinación
        - i: Indice del registro de la base de datos
    
'''
def recurseCombi(re,l,i):
    #Comporbamos si hemos llegado a hacer todas las combinaciones con los nodos de l
    if len(l) == 0:
        lk = len(re)
        k1 = list(re)
        #Comprobamos se hay duplicacion de nodos es esta combinación
        if duplicatedInList(k1) == False:
            #Obtenemos la transacción
            trans = data1[i]
            #Obtenemos los tiempos
            for z in range(lk):
                e = k1[z]
                c = trans[e]
                l.append(c)
            #Guardamos al combinación de tiempos
            combis.append(l)
    else:
        for j in l[0]:
            recurseCombi(re+[j],l[1:],i)

'''
    Nombre de la función: getConstraints
    Funcionalidad: Calcular las restricciones temporales de los chronicle
    Parametros:
        - data1: Lista de los registros con los nodos en modo número.
        - val: Diccionario con las correspondecias letra-número de los nodos.
        - flat_list: Lista de frozenset con los nodos de cada registra en versión numerica
'''
def getConstraints(data1,val,flat_list):

    all_combis = []
    for patron, freq in itemsets.items():
        lista_trans = []
        len_patron = len(patron)

        #Genero el grafo a partir del patron considerando constraints infinitos
        patron1 = list(patron)
        
        #Estructura para el chronicle
        lf = [[patron1[z],patron1[z1],math.inf, -math.inf] for z in range(len_patron) for z1 in range(z+1,len_patron)]

        #Traducción de letra a numero de los nodos
        for l1 in lf:
            found=0
            for key,value in val.items():
                if l1[0] in value:
                    s = ''
                    ind = value.index(l1[0])
                    if ind != 0: s=str(ind)
                    l1[0] = key+s
                    found+=1
                if l1[1] in value:
                    s = ''
                    ind = value.index(l1[1])
                    if ind != 0: s=str(ind)
                    l1[1] = key+s
                    found+=1
                if found==2: break


        all_combis.clear()
        m=0

        t = time()

        #Recorro las transaccciones para ver cuales cumplen el patron y obtengo todas las combinaciones de tiempos de cada transaccion
        for transaccion in flat_list:
            #Comporbar si el patron es un subset de la transacción
            if patron.issubset(transaccion):
            
                lextend=[[w for v in val.values() for w in v if i in v and w in transaccion] for i in patron1]
                combis.clear()
                #Obtenemos todas las combinaciones de tiempos para el registro m
                recurseCombi([],lextend,m)
                #Introducimos las combinaciones en all_combis
                all_combis.append(list(combis))
            m+=1


        #Obtengo todas las combinaciones posibles considerando todas las transacciones para ver qué combinacion es mejor
        all_combis = list(itertools.product(*all_combis))

        all_combis_sets = []
        combis_sets=[] # Todas las posibles difernecias entre los nodos
        combis_len = len(all_combis[0][0])
        conjunto = set()
        minimum = math.inf
        
        #Inicializo los sets
        nCombi = int(math.factorial(combis_len)/(2*math.factorial(combis_len-2)))
        r=[]
        for k in all_combis:
            z = 0
            
            #Calculamos todas las posibles difernecias entre los nodos
            for i in range(combis_len):
                for j in range(i+1, combis_len):
                    combis_sets.append(set())
                    for l in k:
                        combis_sets[z].add(l[j]-l[i])
                    z+=1

            sum=0
            #Guardamos el set de diferencias con minima diferencia entre sus datos
            for c in combis_sets:
                sum+= max(c)-min(c)
            if sum < minimum:
                minimum = sum
                r = combis_sets
            summatory= 0
            combis_sets=[]

        z = 0
        #Guardamos las mejores restricciones de tiempo en la estructura del Chronicle
        for i in lf:
            i[2] = min(r[z])
            i[3] = max(r[z])
            z+=1
        
        t1 = time()-t

        print(lf," Sup: ", freq)


if __name__ == "__main__":
    
    #nombre_bd = BDExample
    nombre_bd = input("Introduzca el nombre de la base de datos: ")
    data,flat_list,val,data1 = loadBD(nombre_bd+".txt")
    
    itemsets={}
    constraints=[]
    combis = []
    #minSup = 6
    minSup = int(input("Introduzca el soporte mínimo para los Chronicles: "))
    #minLen = 3
    minLen = int(input("Introduzca el tamaño mínimo para los Chronicles: "))
    if(minSup > 0 and minLen > 1):
        LCM(flat_list, minSup, minLen)
        data2 = {}
        val2 = {}

        getConstraints(data1,val,flat_list)
    else:
        print("Imposible crear Chronicles con los datos especificados")

        